# Proteomics Data Analysis

## Libraries

In [ ]:
suppressPackageStartupMessages({
  library(conflicted)
  library(tidyverse)
  library(ggVennDiagram)
  library(patchwork)
  library(scales)
  library(IRdisplay)
  library(sysfonts)
  library(showtext)
})

## Settings

In [ ]:
# Graphics
default_dpi <- 600
default_width <- 80 # 50-80 to 120-170
default_height <- 66
default_unit <- "mm"
default_font_size <- 8
default_text_color <- "#231F20"
default_label_characters <- 20

showtext_opts(dpi = default_dpi)
showtext_auto()

theme_set(
  theme_minimal() +
    theme(
      text = element_text(size = default_font_size, color = default_text_color),
      plot.title = element_text(hjust = 0.5),
      plot.title.position = "panel",
      plot.subtitle = element_text(hjust = 0.5),
      plot.tag = element_text(size = default_font_size, color = default_text_color, face = "bold"),
      axis.ticks = element_line(color = default_text_color),
      legend.title = element_text(size = 0.8 * default_font_size),
      legend.text = element_text(size = 0.5 * default_font_size),
      legend.key.size = unit(4, "mm"),
      panel.grid = element_line(color = "#ECE9EA", linewidth = 0.25),
      panel.grid.minor = element_line(linewidth = 0.125, linetype = 2),
      panel.background = element_rect(fill = "transparent", color = NA),
      panel.border = element_rect(color = default_text_color, fill = NA),
      plot.background = element_rect(fill = "transparent", color = NA),
      axis.text = element_text(size = 6),
    )
)
update_geom_defaults("text", list(size = 0.8 * default_font_size / .pt))

# Statistics
lfq_fdr_cutoff <- 0.05
lbq_fdr_cutoff <- 0.01

# Labels
conditionLabels <- c("CTRL vs. DMSO",
                     "EE vs. DMSO",
                     "LNG vs. DMSO",
                     "EE + LNG vs. DMSO",
                     "S-23 vs. DMSO")
names(conditionLabels) <- c("CTRL_vs_DMSO",
                            "EE_vs_DMSO",
                            "LNG_vs_DMSO",
                            "EE+LNG_vs_DMSO",
                            "S-23_vs_DMSO")

# Directories
dirs <- list("graphs", "results", "logs")

for (dir in dirs) {
  if (dir.exists(dir)) {
    unlink(dir, recursive = TRUE)
  }
  dir.create(dir, recursive = TRUE)
}

## Global Helper Functions

In [ ]:
source("helpers/global.r")

## Load Data

The PSMs identified by Proteome Discoverer are read and filtered to remove PSMs with high isolation interference, PSMs matching mutliple protein groups and PSMs matching a contaminant protein.

For this analysis, only human keratins were selected as contaminants. Therefore the [list of all human keratins](https://www.uniprot.org/uniprotkb?query=keratin+human&facets=reviewed%3Atrue%2Cmodel_organism%3A9606%2Cexistence%3A1) was downloaded from UniProtKB/Swiss-Prot.

In [ ]:
source("helpers/load_data.r")

In [ ]:
lfqPsm_originalData <- load_data("data/LFQ/LFQ_PSMs.tsv", "data/contaminants.tsv", type = "LFQ")

In [ ]:
lfqAnnotation <- read.table("data/LFQ/annotation.tsv", sep = "\t", header = TRUE)

In [ ]:
lbqPsm_originalData <- load_data("data/LBQ/LBQ_PSMs.tsv", "data/contaminants.tsv", type = "LBQ")

In [ ]:
if (!file.exists("data/LBQ/annotation.tsv")) {
  source("helpers/lbq_annotation.R")
  create_lbq_annotation_file("annotation.tsv", "./data/LBQ/")
}

lbqAnnotation <- read.table("data/LBQ/annotation.tsv", sep = "\t", header = TRUE)

## Labeling Efficiency and Uniformity

In [ ]:
source("helpers/labeling.R")

In [ ]:
labeling_efficiency <- get_labeling_efficiency(lbqPsm_originalData)

cat(labeling_efficiency)

In [ ]:
reporterIon_distribution <- get_reporterIon_distribution(lbqPsm_originalData)
reporterIon_stats <- get_reporterIon_stats(reporterIon_distribution)

reporterIon_density_plots <- get_reporterIon_density_plots(reporterIon_distribution, reporterIon_stats)

save_and_show_plot(
  "reporterIon_density_plots",
  reporterIon_density_plots,
  plot_width = 120,
  plot_height = 75
)

## Convert LFQ PD Data to MSstatsFormat

In [ ]:
lfqPsm_formattedData <- MSstats::PDtoMSstatsFormat(
  lfqPsm_originalData %>% mutate(Spectrum.File = File.ID),
  annotation = lfqAnnotation,
  which.quantification = "Precursor.Abundance",
  which.proteinid = "Master.Protein.Accessions",
  use_log_file = TRUE,
  log_file_path = "logs/PDtoMSstatsFormat.log"
)

# Ensure the correct file arrangement
run_levels <- paste0("F", sort(as.numeric(gsub(
  "F", "", unique(lfqPsm_formattedData$Run)
))))

lfqPsm_formattedData <- lfqPsm_formattedData %>%
  as.data.frame() %>%
  mutate(Missing = is.na(Intensity),
         Run = factor(Run, levels = run_levels))

### Basic Information

#### Total PSMs

In [ ]:
cat(nrow(lfqPsm_originalData))

#### Total Peptide ions

In [ ]:
cat(length(unique(lfqPsm_formattedData %>% pull("PeptideModifiedSequence"))))

#### Missing Data

In [ ]:
tmp <- lfqPsm_formattedData %>%
  dplyr::filter(Missing == TRUE)

cat("\nPer Run")
table(tmp$Run)

cat("\nPer Condition")
table(tmp$Condition)

## Convert LBQ PD Data to MSstatsTMTFormat

In [ ]:
lbqPsm_formattedData <- MSstatsTMT::PDtoMSstatsTMTFormat(
  # Only use the identification node with a static TMTpro modification
  lbqPsm_originalData %>% dplyr::filter(Identifying.Node.No == 4),
  annotation = lbqAnnotation,
  which.proteinid = "Master.Protein.Accessions",
  use_log_file = TRUE,
  log_file_path = "logs/PDtoMSstatsTMTFormat.log"
)

# Ensure the correct channel arrangement
channel_levels <- c(
  "126",
  "127N",
  "127C",
  "128N",
  "128C",
  "129N",
  "129C",
  "130N",
  "130C",
  "131N",
  "131C",
  "132N",
  "132C",
  "133N",
  "133C",
  "134N"
)

lbqPsm_formattedData <- lbqPsm_formattedData %>%
  as.data.frame() %>%
  mutate(Missing = is.na(Intensity)) %>%
  mutate(Channel = factor(Channel, levels = channel_levels))

### Basic Information

#### Total PSMs

In [ ]:
cat(nrow(lbqPsm_originalData))

#### Total Peptide ions

In [ ]:
cat(length(unique(lbqPsm_formattedData %>% pull("PeptideSequence"))))

#### Missing Data

In [ ]:
tmp <- lbqPsm_formattedData %>%
  dplyr::filter(Missing == TRUE)

cat("Missing Values in LBQ Data:\n")

cat("\nPer Channel")
table(tmp$Channel)

cat("\nPer Condition")
table(tmp$Condition)

cat("\nPer Tech. Replicate")
table(tmp$TechRepMixture)

## Plot Missing Values

### Visualize the Missing Values in the Original Dataset

In [ ]:
source("helpers/missing_values.R")

In [ ]:
missing_data_peptide <- get_missing_data(lfqPsm_formattedData, lbqPsm_formattedData)
missingness_data <- get_missingness_data(missing_data_peptide)
missing_summary_data <- get_missing_summary_data(lfqPsm_formattedData, lbqPsm_formattedData)

missingValuesPlot_original <- get_complete_missingness_plot(missingness_data, missing_data_peptide, missing_summary_data)

save_and_show_plot(
  "missingValuesPlot_original",
  missingValuesPlot_original,
  plot_width = 120
)

## Filter Out Proteins with Too Much Missingness

In [ ]:
lfqPsm_formattedData_filtered <- lfqPsm_formattedData %>%
  group_by(ProteinName) %>%
  dplyr::filter(mean(is.na(Intensity)) < 0.5)

cat(
  sprintf(
    "LFQ:\nBefore filtering: %s proteins\nAfter filtering: %s proteins\nRemoved proteins: %s\n\n",
    length(unique(lfqPsm_formattedData$ProteinName)),
    length(unique(
      lfqPsm_formattedData_filtered$ProteinName
    )),
    length(unique(lfqPsm_formattedData$ProteinName)) - length(unique(
      lfqPsm_formattedData_filtered$ProteinName
    ))
  )
)

lbqPsm_formattedData_filtered <- lbqPsm_formattedData %>%
  group_by(ProteinName) %>%
  dplyr::filter(mean(is.na(Intensity)) < 0.5) %>%
  ungroup()

cat(
  sprintf(
    "LBQ:\nBefore filtering: %s proteins\nAfter filtering: %s proteins\nRemoved proteins: %s\n\n",
    length(unique(lbqPsm_formattedData$ProteinName)),
    length(unique(
      lbqPsm_formattedData_filtered$ProteinName
    )),
    length(unique(lbqPsm_formattedData$ProteinName)) - length(unique(
      lbqPsm_formattedData_filtered$ProteinName
    ))
  )
)

## Re-Plot Missing Values

In [ ]:
missing_data <- get_missing_data(lfqPsm_formattedData_filtered,
                                 lbqPsm_formattedData_filtered)
missingness_data <- get_missingness_data(missing_data)
missing_summary_data <- get_missing_summary_data(lfqPsm_formattedData_filtered,
                                                 lbqPsm_formattedData_filtered)

missingValuesPlot_filtered <- get_complete_missingness_plot(missingness_data, missing_data, missing_summary_data)

save_and_show_plot(
  "missingValuesPlot_filtered",
  missingValuesPlot_filtered,
  plot_width = 120
)

## Prepare Data for Processing

Both, `MSstatsPrepareForDataProcess()` and `MSstatsPrepareForSummarizationTMT()` are internal methods of `MSstats` and `MSstatsTMT`, respectively. They are called in the process of `dataProcess()` (`MSstats`) and `proteinSummarization()` (`MSstatsTMT`). Here, they are called outside of these methods as they provide the data just before normalization which is required to plot the effect of said normalization.

In [ ]:
MSstatsConvert::MSstatsLogsSettings(
  use_log_file = TRUE,
  append = FALSE,
  verbose = TRUE,
  base = "logs/MSstats_prepareDataProcess_log_",
  pkg_name = "MSstats"
)

lfqData_beforeSummarization <- MSstats:::MSstatsPrepareForDataProcess(
  # Default parameters used in dataProcess (https://github.com/Vitek-Lab/MSstats/blob/devel/R/dataProcess.R)
  lfqPsm_formattedData_filtered,
  log_base = 2,
  fix_missing = NULL
)

In [ ]:
MSstatsConvert::MSstatsLogsSettings(
  use_log_file = TRUE,
  append = FALSE,
  verbose = TRUE,
  base = "logs/MSstatsTMT_prepareSummarization_log_",
  pkg_name = "MSstatsTMT"
)

lbqData_beforeSummarization <- MSstatsTMT:::MSstatsPrepareForSummarizationTMT(
  # Default parameters used in proteinSummarization (https://github.com/Vitek-Lab/MSstatsTMT/blob/devel/R/proteinSummarization.R)
  lbqPsm_formattedData_filtered,
  method = "msstats",
  global_norm = TRUE,
  reference_norm = FALSE,
  remove_norm_channel = TRUE,
  remove_empty_channel = TRUE,
  MBimpute = TRUE,
  maxQuantileforCensored = NULL
)

## Perform MSstats Data Processing

In [ ]:
# Default parameters used in dataProcess() (https://github.com/Vitek-Lab/MSstats/blob/devel/R/dataProcess.R)
#
# raw,
# logTrans = 2,
# normalization = "equalizeMedians",
# nameStandards = NULL,
# featureSubset = "all",
# remove_uninformative_feature_outlier = FALSE,
# min_feature_count = 2,
# n_top_feature = 3,
# summaryMethod = "TMP",
# equalFeatureVar = TRUE,
# censoredInt = "NA",
# MBimpute = TRUE,
# remove50missing = FALSE,
# fix_missing = NULL,
# maxQuantileforCensored = 0.999,
# use_log_file = TRUE,
# append = FALSE,
# verbose = TRUE,
# log_file_path = NULL,
# numberOfCores = 1

lfqData_afterSummarization <- MSstats::dataProcess(lfqPsm_formattedData_filtered,
                                                   use_log_file = TRUE,
                                                   log_file_path = "logs/dataProcess.log")

lfqData_afterSummarization$ProteinLevelData <- lfqData_afterSummarization$ProteinLevelData %>%
  mutate(originalRUN = factor(originalRUN, levels = run_levels))

In [ ]:
# Default parameters used in proteinSummarization() (https://github.com/Vitek-Lab/MSstatsTMT/blob/devel/R/proteinSummarization.R)
#
# data,
# method = 'msstats',
# global_norm = TRUE,
# reference_norm = TRUE,
# remove_norm_channel = TRUE,
# remove_empty_channel = TRUE,
# MBimpute = TRUE,
# maxQuantileforCensored = NULL,
# use_log_file = TRUE,
# append = FALSE,
# verbose = TRUE,
# log_file_path = NULL,
# msstats_log_path = NULL

lbqData_afterSummarization <- MSstatsTMT::proteinSummarization(
  lbqPsm_formattedData_filtered,
  reference_norm = FALSE,
  use_log_file = TRUE,
  log_file_path = "logs/proteinSummarization.log"
)

## Visualize Normalization Effect

In [ ]:
source("helpers/normalization.R")

In [ ]:
lfq_abundance_density_data <- get_abundance_density_data(
  lfqData_beforeSummarization,
  lfqData_afterSummarization$FeatureLevelData,
  originalRUN,
  originalRUN,
  run_levels,
  ABUNDANCE,
  newABUNDANCE,
  "LFQ"
)
lbq_abundance_density_data <- get_abundance_density_data(
  lbqData_beforeSummarization,
  lbqData_afterSummarization$FeatureLevelData,
  Channel,
  Channel,
  channel_levels,
  log2Intensity,
  log2Intensity,
  "LBQ"
)

abundance_density_combined <- get_complete_abundance_density_plot(lfq_abundance_density_data, lbq_abundance_density_data)

save_and_show_plot(
  "abundance_density_combined",
  abundance_density_combined,
  plot_width = 120,
  plot_height = 100
)

## Dynamic Range

In [ ]:
source("helpers/dynamic_range.R")

In [ ]:
dynamicRange_data <- get_dynamic_range_data(
  lfqData_afterSummarization$ProteinLevelData,
  lbqData_afterSummarization$ProteinLevelData
)
dynamicRange_plot <- get_dynamic_range_plot(dynamicRange_data)

save_and_show_plot(
  "dynamicRange_plot",
  dynamicRange_plot,
  plot_width = 80,
  plot_height = 50
)

In [ ]:
lfq <- dynamicRange_data %>% dplyr::filter(Source == "LFQ") %>% pull(Abundance)
lbq <- dynamicRange_data %>% dplyr::filter(Source == "LBQ") %>% pull(Abundance)

lfq_min <- min(lfq)
lfq_max <- max(lfq)
cat(sprintf("Difference between the min and the max: %.2f\n", lfq_max -
              lfq_min))

lfq_q1 <- quantile(lfq, 0.05, na.rm = TRUE)
lfq_q3 <- quantile(lfq, 0.95, na.rm = TRUE)
cat(
  sprintf(
    "Difference between the 5 %% quantile and the 95 %% quantile: %.2f\n\n",
    lfq_q3 - lfq_q1
  )
)

lbq_min <- min(lbq)
lbq_max <- max(lbq)
cat(sprintf("Difference between the min and the max: %.2f\n", lbq_max -
              lbq_min))

lbq_q1 <- quantile(lbq, 0.05, na.rm = TRUE)
lbq_q3 <- quantile(lbq, 0.95, na.rm = TRUE)
cat(
  sprintf(
    "Difference between the 5 %% quantile and the 95 %% quantile: %.2f\n\n",
    lbq_q3 - lbq_q1
  )
)

## Group comparisons

In [ ]:
source("helpers/group_comparison.R")

In [ ]:
comparison_matrix <- matrix(
  c(1, 0, 0, 0, 0, -1,  # EE vs DMSO
    0, 1, 0, 0, 0, -1,  # LNG vs DMSO
    0, 0, 1, 0, 0, -1,  # EE+LNG vs DMSO
    0, 0, 0, 1, 0, -1,  # S-23 vs DMSO
    0, 0, 0, 0, 1, -1   # CTRL vs DMSO
  ),
  nrow = 5, byrow = TRUE
)

rownames(comparison_matrix) <- c("EE_vs_DMSO",
                                 "LNG_vs_DMSO",
                                 "EE+LNG_vs_DMSO",
                                 "S-23_vs_DMSO",
                                 "CTRL_vs_DMSO")
colnames(comparison_matrix) <- c("EE", "LNG", "EE+LNG", "S-23", "CTRL", "DMSO")

In [ ]:
# Default parameters used in groupComparison() (https://github.com/Vitek-Lab/MSstats/blob/devel/R/groupComparison.R)
#
# contrast.matrix,
# data,
# save_fitted_models = TRUE,
# log_base = 2,
# use_log_file = TRUE,
# append = FALSE,
# verbose = TRUE,
# log_file_path = NULL,
# numberOfCores = 1

lfqModel <- MSstats::groupComparison(
  contrast.matrix = comparison_matrix,
  data = lfqData_afterSummarization,
  use_log_file = TRUE,
  log_file_path = "logs/groupComparison.log"
)

In [ ]:
# Default parameters used in groupComparisonTMT() (https://github.com/Vitek-Lab/MSstatsTMT/blob/devel/R/groupComparisonTMT.R)
#
# data,
# contrast.matrix = "pairwise",
# moderated = FALSE,
# adj.method = "BH",
# remove_norm_channel = TRUE,
# remove_empty_channel = TRUE,
# save_fitted_models = FALSE,
# use_log_file = TRUE,
# append = FALSE,
# verbose = TRUE,
# log_file_path = NULL

lbqModel <- MSstatsTMT::groupComparisonTMT(
  contrast.matrix = comparison_matrix,
  data = lbqData_afterSummarization,
  moderated = TRUE,
  use_log_file = TRUE,
  log_file_path = "logs/groupComparisonTMT.log"
)

In [ ]:
lfqModel$ComparisonResult <- lfqModel$ComparisonResult %>%
  dplyr::filter(!is.infinite(log2FC)) %>%
  dplyr::filter(!is.na(log2FC)) %>%
  mutate(LabelFactor = factor(Label, levels = names(conditionLabels)))

In [ ]:
lbqModel$ComparisonResult <- lbqModel$ComparisonResult %>%
  dplyr::filter(!is.infinite(log2FC)) %>%
  dplyr::filter(!is.na(log2FC)) %>%
  mutate(LabelFactor = factor(Label, levels = names(conditionLabels)))

### DAP Overlap

In [ ]:
venn_diagrams <- get_venn_diagram_plot(
  lfqModel$ComparisonResult,
  lbqModel$ComparisonResult,
  lfqData_afterSummarization$ProteinLevelData,
  lbqData_afterSummarization$ProteinLevelData
)

save_and_show_plot(
  "venn_diagrams",
  venn_diagrams,
  plot_width = 120,
  plot_height = 77
)

### Fold Change Density

In [ ]:
lfq_fold_change_stats <- get_fold_change_stats(lfqModel$ComparisonResult)
lbq_fold_change_stats <- get_fold_change_stats(lbqModel$ComparisonResult)

combined_fold_change_density <- get_combined_fold_change_density_plot(
  lfqModel$ComparisonResult,
  lfq_fold_change_stats,
  lbqModel$ComparisonResult,
  lbq_fold_change_stats
)

save_and_show_plot(
  "combined_foldChangeDensity",
  combined_fold_change_density,
  plot_width = 170,
  plot_height = 90
)

### Volcano Plots

In [ ]:
combined_volcanoPlots <- get_combined_volcano_plot(
  lfqModel$ComparisonResult,
  lfq_fdr_cutoff,
  lbqModel$ComparisonResult,
  lbq_fdr_cutoff
)

save_and_show_plot(
  "combined_volcanoPlots",
  combined_volcanoPlots,
  plot_width = 170,
  plot_height = 90
)

In [ ]:
lfq_significance_barchart_data <- get_significant_proteins_amount_data(lfqModel$ComparisonResult, lfq_fdr_cutoff)
lbq_significance_barchart_data <- get_significant_proteins_amount_data(lbqModel$ComparisonResult, lbq_fdr_cutoff)

combined_significance_barchart <- get_combined_significance_barchart(lfq_significance_barchart_data,
                                                                     lbq_significance_barchart_data)

save_and_show_plot(
  "combined_significanceBarchart",
  combined_significance_barchart,
  plot_width = 170,
  plot_height = 90
)

### Filter Proteins based on log2FC

In [ ]:
lfqDap_subset <- get_log2fc_and_pvalue_filtered_data(lfqModel$ComparisonResult,
                                                     lfq_fold_change_stats,
                                                     lfq_fdr_cutoff)

lbqDap_subset <- get_log2fc_and_pvalue_filtered_data(lbqModel$ComparisonResult,
                                                     lbq_fold_change_stats,
                                                     lbq_fdr_cutoff)

### Filtered DAP Overlap

In [ ]:
filtered_venn_diagrams <- get_venn_diagram_plot(lfqDap_subset, lbqDap_subset)

save_and_show_plot(
  "filtered_venn_diagrams",
  filtered_venn_diagrams,
  plot_width = 120,
  plot_height = 77
)

### Principal Component Analysis

In [ ]:
source("helpers/pca.R")

#### All DAPs

In [ ]:
lfqData_pcaData <- get_pca_data(lfqData_afterSummarization$ProteinLevelData,
                                lfqAnnotation,
                                "LFQ")

In [ ]:
lfqPcaPlot_1.2_plots <- get_pca_plots(lfqData_pcaData, PC1, PC2, "LFQ")

save_and_show_plot(
  "lfqPcaPlot_1.2_plots",
  lfqPcaPlot_1.2_plots,
  plot_width = 60
)

In [ ]:
lbqData_pcaData <- get_pca_data(lbqData_afterSummarization$ProteinLevelData,
                                lbqAnnotation,
                                "LBQ")

In [ ]:
lbqPcaPlot_1.2_plots <- get_pca_plots(lbqData_pcaData, PC1, PC2, "LBQ")

save_and_show_plot(
  "lbqPcaPlot_1.2_plots",
  lbqPcaPlot_1.2_plots,
  plot_width = 110
)

In [ ]:
lbqPcaPlot_3.4_plots <- get_pca_plots(lbqData_pcaData, PC3, PC4, "LBQ")

save_and_show_plot(
  "lbqPcaPlot_3.4_plots",
  lbqPcaPlot_3.4_plots,
  plot_width = 120
)

## Method Comparison

In [ ]:
combined_dap_correlation_plots <- get_combined_correlation_scatter_plot(
  lfqModel$ComparisonResult,
  lbqModel$ComparisonResult,
  lfq_fdr_cutoff,
  lbq_fdr_cutoff,
  lfqDap_subset,
  lbqDap_subset,
  lfq_fdr_cutoff,
  lbq_fdr_cutoff
)

save_and_show_plot(
  "combined_dap_correlation_plots",
  combined_dap_correlation_plots,
  plot_width = 120,
  plot_height = 95
)

### DAP per Condition

In [ ]:
lbqDap_subset %>%
  group_by(LabelFactor) %>%
  summarize(No = n())

#### Removal of DAPs present in CTRL vs. DMSO from other contrasts

In [ ]:
ctrl_dmso_proteins <- lbqDap_subset %>%
  group_by(LabelFactor) %>%
  dplyr::filter(LabelFactor == "CTRL_vs_DMSO") %>%
  pull(Protein)

lbqDap_subset <- lbqDap_subset %>%
  dplyr::filter(!Protein %in% ctrl_dmso_proteins)

#### Overlap

In [ ]:
dapOverlap_allConditions_vennPlot <- get_condition_overlap_venn_diagram_plot(lbqDap_subset)
save_and_show_plot(
  "dapOverlap_allConditions_vennPlot",
  dapOverlap_allConditions_vennPlot,
  plot_width = 50,
  plot_height = 35
)

lbqDap_subset %>% group_by(LabelFactor) %>% summarize(No = n())

## Enrichment Analysis

In [ ]:
source("helpers/enrichment_analysis.R")

In [ ]:
background_gene_list <- get_genes_from_proteins(lbqModel$ComparisonResult)

In [ ]:
# Only use the top-200 proteins per contrast for enrichment analyses
lbq_enrichment_data <- lbqDap_subset %>%
  group_by(Label) %>%
  arrange(adj.pvalue) %>%
  slice_head(n = 200) %>%
  ungroup

In [ ]:
lbq_enrichment_data_vennPlot <- get_condition_overlap_venn_diagram_plot(lbq_enrichment_data)

save_and_show_plot(
  "lbq_enrichment_data_vennPlot",
  lbq_enrichment_data_vennPlot,
  plot_width = 50,
  plot_height = 35
)

In [ ]:
# Print the top-200 proteins per contrast incl. Proteome Discoverer Metadata

# Read PD Protein Metadata
lbqProteins <- read.table("data/LBQ/LBQ_Proteins.tsv",
                          sep = "\t",
                          header = TRUE)

lbqProteins <- lbqProteins %>%
  select(
    Protein = Accession,
    Gene = Gene.Symbol,
    q_value = Exp.q.value.Combined,
    Coverage = Coverage.in.Percent,
    PSMs = Number.of.PSMs,
    Peptides = Number.of.Peptides,
    Unique_Peptides = Number.of.Unique.Peptides,
    XCorr = Score.Sequest.HT.Sequest.HT
  )

ee_dmso_table <- get_formatted_top_proteins_per_label(lbq_enrichment_data, lbqProteins, "EE_vs_DMSO")
save_and_show_table("top200_EEvsDMSO", ee_dmso_table)

lng_dmso_table <- get_formatted_top_proteins_per_label(lbq_enrichment_data, lbqProteins, "LNG_vs_DMSO")
save_and_show_table("top200_LNGvsDMSO", lng_dmso_table)

ee.lng_dmso_table <- get_formatted_top_proteins_per_label(lbq_enrichment_data, lbqProteins, "EE+LNG_vs_DMSO")
save_and_show_table("top200_EE+LNGvsDMSO", ee.lng_dmso_table)

s23_dmso_table <- get_formatted_top_proteins_per_label(lbq_enrichment_data, lbqProteins, "S-23_vs_DMSO")
save_and_show_table("top200_S-23vsDMSO", s23_dmso_table)

lbq_enrichment_table <- bind_rows(ee_dmso_table,
                                  lng_dmso_table,
                                  ee.lng_dmso_table,
                                  s23_dmso_table) %>%
  select(-c("log2 FC", "adj. p-value")) %>%
  distinct()
  
save_and_show_table("enrichment_proteins", lbq_enrichment_table)

In [ ]:
regulated_gene_list_ee.lng.eelng.s23 <- get_intersection_genes(lbq_enrichment_data, lbq_fdr_cutoff, "EE/LNG/EE+LNG/S-23")
regulated_gene_list_lng.eelng.s23 <- get_intersection_genes(lbq_enrichment_data, lbq_fdr_cutoff, "LNG/EE+LNG/S-23")

regulated_gene_list_ee_all <- get_genes_from_proteins(lbq_enrichment_data %>% dplyr::filter(Label == "EE_vs_DMSO"))
regulated_gene_list_lng_all <- get_genes_from_proteins(lbq_enrichment_data %>% dplyr::filter(Label == "LNG_vs_DMSO"))
regulated_gene_list_eelng_all <- get_genes_from_proteins(lbq_enrichment_data %>% dplyr::filter(Label == "EE+LNG_vs_DMSO"))
regulated_gene_list_s23_all <- get_genes_from_proteins(lbq_enrichment_data %>% dplyr::filter(Label == "S-23_vs_DMSO"))

### Gene Ontology

#### Intersection of EE, LNG, EE +LNG, and S-23

In [ ]:
go_ee.lng.eelng.s23 <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_ee.lng.eelng.s23,
  background_gene_list,
  "EE/LNG/EE+LNG/S-23",
  "ee.lng.eelng.s23"
)

#### Intersection of LNG, EE + LNG, and S-23

In [ ]:
go_lng.eelng.s23 <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_lng.eelng.s23,
  background_gene_list,
  "LNG/EE+LNG/S-23",
  "lng.eelng.s23"
)

#### EE

In [ ]:
go_ee <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_ee_all,
  background_gene_list,
  "EE vs. DMSO",
  "ee_all"
)

#### LNG

In [ ]:
go_lng <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_lng_all,
  background_gene_list,
  "LNG vs. DMSO",
  "lng_all"
)

#### EE + LNG

In [ ]:
go_eelng <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_eelng_all,
  background_gene_list,
  "EE+LNG vs. DMSO",
  "eelng_all"
)

#### S-23

In [ ]:
go_s23 <- perform_enrichment_analysis(
  "GO",
  regulated_gene_list_s23_all,
  background_gene_list,
  "S-23 vs. DMSO",
  "s23_all"
)

#### Bubble Plot

In [ ]:
go_results <- list(
  go_ee.lng.eelng.s23,
  go_lng.eelng.s23,
  go_ee,
  go_lng,
  go_eelng,
  go_s23
)
go_bubbleplot <- get_enrichment_bubbleplot(go_results)

save_and_show_plot(
  "go_bubbleplot",
  go_bubbleplot,
  plot_width = 120,
  plot_height = 100
)

### Kyoto Encyclopedia of Genes and Genomes

#### Intersection of EE, LNG, EE +LNG, and S-23

In [ ]:
kegg_ee.lng.eelng.s23 <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_ee.lng.eelng.s23,
  background_gene_list,
  "EE/LNG/EE+LNG/S-23",
  "ee.lng.eelng.s23"
)

#### Intersection of LNG, EE + LNG, and S-23

In [ ]:
kegg_lng.eelng.s23 <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_lng.eelng.s23,
  background_gene_list,
  "LNG/EE+LNG/S-23",
  "lng.eelng.s23"
)

#### EE

In [ ]:
kegg_ee <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_ee_all,
  background_gene_list,
  "EE vs. DMSO",
  "ee_all"
)

#### LNG

In [ ]:
kegg_lng <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_lng_all,
  background_gene_list,
  "LNG vs. DMSO",
  "lng_all"
)

#### EE + LNG

In [ ]:
kegg_eelng <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_eelng_all,
  background_gene_list,
  "EE+LNG vs. DMSO",
  "eelng_all"
)

#### S-23

In [ ]:
kegg_s23 <- perform_enrichment_analysis(
  "KEGG",
  regulated_gene_list_s23_all,
  background_gene_list,
  "S-23 vs. DMSO",
  "s23_all"
)

#### Bubble Plot

In [ ]:
kegg_results <- list(
  kegg_ee.lng.eelng.s23,
  kegg_lng.eelng.s23,
  kegg_ee,
  kegg_lng,
  kegg_eelng,
  kegg_s23
)
kegg_bubbleplot <- get_enrichment_bubbleplot(kegg_results)

save_and_show_plot(
  "kegg_bubbleplot",
  kegg_bubbleplot,
  plot_width = 120,
  plot_height = 100
)

### Disease Ontology Semantic and Enrichment analysis

#### Intersection of EE, LNG, EE +LNG, and S-23

In [ ]:
dose_ee.lng.eelng.s23 <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_ee.lng.eelng.s23,
  background_gene_list,
  "EE/LNG/EE+LNG/S-23",
  "ee.lng.eelng.s23"
)

#### Intersection of LNG, EE + LNG, and S-23

In [ ]:
dose_lng.eelng.s23 <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_lng.eelng.s23,
  background_gene_list,
  "LNG/EE+LNG/S-23",
  "lng.eelng.s23"
)

#### EE

In [ ]:
dose_ee <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_ee_all,
  background_gene_list,
  "EE vs. DMSO",
  "ee_all"
)

#### LNG

In [ ]:
dose_lng <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_lng_all,
  background_gene_list,
  "LNG vs. DMSO",
  "lng_all"
)

#### EE + LNG

In [ ]:
dose_eelng <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_eelng_all,
  background_gene_list,
  "EE+LNG vs. DMSO",
  "eelng_all"
)

#### S-23

In [ ]:
dose_s23 <- perform_enrichment_analysis(
  "DOSE",
  regulated_gene_list_s23_all,
  background_gene_list,
  "S-23 vs. DMSO",
  "s23_all"
)

#### Bubble Plot

In [ ]:
highlighted_terms <- c("Major Depressive Disorder",
                       "Unipolar Depression",
                       "Bipolar Depression")

dose_results <- list(
  dose_ee.lng.eelng.s23,
  dose_lng.eelng.s23,
  dose_ee,
  dose_lng,
  dose_eelng,
  dose_s23
)
dose_bubbleplot <- get_enrichment_bubbleplot(dose_results, highlighted_terms)

save_and_show_plot(
  "dose_bubbleplot",
  dose_bubbleplot,
  plot_width = 120,
  plot_height = 100
)

### DISGENET

#### Intersection of EE, LNG, EE +LNG, and S-23

In [ ]:
disgenet_ee.lng.eelng.s23 <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_ee.lng.eelng.s23,
  background_gene_list,
  "EE/LNG/EE+LNG/S-23",
  "ee.lng.eelng.s23"
)

#### Intersection of LNG, EE + LNG, and S-23

In [ ]:
disgenet_lng.eelng.s23 <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_lng.eelng.s23,
  background_gene_list,
  "LNG/EE+LNG/S-23",
  "lng.eelng.s23"
)

#### EE

In [ ]:
disgenet_ee <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_ee_all,
  background_gene_list,
  "EE vs. DMSO",
  "ee_all"
)

#### LNG

In [ ]:
disgenet_lng <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_lng_all,
  background_gene_list,
  "LNG vs. DMSO",
  "lng_all"
)

#### EE + LNG

In [ ]:
disgenet_eelng <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_eelng_all,
  background_gene_list,
  "EE+LNG vs. DMSO",
  "eelng_all"
)

#### S-23

In [ ]:
disgenet_s23 <- perform_enrichment_analysis(
  "DISGENET",
  regulated_gene_list_s23_all,
  background_gene_list,
  "S-23 vs. DMSO",
  "s23_all"
)

#### Bubble Plot

In [ ]:
highlighted_terms <- c("Major Depressive Disorder",
                       "Unipolar Depression",
                       "Bipolar Disorder")

disgenet_results <- list(
  disgenet_ee.lng.eelng.s23,
  disgenet_lng.eelng.s23,
  disgenet_ee,
  disgenet_lng,
  disgenet_eelng,
  disgenet_s23
)
disgenet_bubbleplot <- get_enrichment_bubbleplot(disgenet_results, highlighted_terms)

save_and_show_plot(
  "disgenet_bubbleplot",
  disgenet_bubbleplot,
  plot_width = 120,
  plot_height = 100
)

## Session

In [ ]:
sessionInfo()